# Collecting Rudyard Kipling's poems

### Dependencies

In [1]:
# General
from pprint import pprint
import string
from string import digits

# to scrape the data from websites
from requests import get
from bs4 import BeautifulSoup

# to make the scraping more human-like by controlling the crawl rate
from IPython.core.display import clear_output
from time import sleep, time
from random import randint

# just in case of errors
from warnings import warn

### Define functions for extracting information from websites

In [2]:
def webscraper(url_list):
    """ Define a function that conducts webscraping """
    # Create an empty list of page soups
    soups = []
    
    # Prepare the loop through the pages
    start_time = time()
    requests = 0
    
    # Iterate through the pages
    for url in url_list:
        if url == "":
            page_soup = ""
                
        else:
            # Make a get request for the contents of the URL
            response = get(url)

            # Pause the loop
            sleep(randint(8,15))

            # Monitor the requests
            requests += 1
            elapsed_time = time() - start_time
            print(f"Request {requests}; Frequency: {requests/elapsed_time} requests/sec")
            clear_output(wait = True) # to replace output with new one (instead of printing many outputs)

            # If there's an error, throw an error warning
            if response.status_code != 200:
                warn(f"Request: {requests}; Status code: {response.status_code}")

            # If there are more requests than expected, break the loop
            if requests > len(url_list):
                warn("Number of requests more than expected.")
                break

            # Scrape with BeautifulSoup
            page_soup = BeautifulSoup(response.text, "html.parser")
            
        soups.append(page_soup)
    
    return soups

In [3]:
def extract_text(poem_list):
    """ Loops through a list of poems and extracts lines of text"""
    poem_lines2 = []
    for poem in poem_list:
        # Create an empty list
        lines = []
        
        # Loop through each poem (a list of lines), extract the text, add to the empty list
        for line in poem:
            lines.append(line.text)
        poem_lines2.append(lines)    
            
    return poem_lines2      

### Webscraping

In [4]:
# Sources of list of Kipling's poems (1885–1918)
url = ["https://www.bartleby.com/364/",
       "https://www.bartleby.com/people/Kipling.html"]